In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy.linalg import pinv2

In [2]:
X_ = pd.read_csv(r'C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\x_2.csv', header = None)
Y_ = pd.read_csv(r'C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\y_2.csv', header = None)
Y_

,0,1,2
0,0.186646,91.275503,7.320161
1,0.158204,91.042365,7.482942
2,0.211328,91.027887,7.586356
3,0.088008,90.827035,7.762458
4,0.103785,90.869638,7.833571
...,...,...,...
126,0.111707,90.667962,8.103729
127,0.126629,90.389588,8.281747
128,0.143911,90.398711,8.354857
129,0.178287,90.096464,8.515829


In [3]:
# onehotencoder = OneHotEncoder(categories='auto')
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X = scaler_x.fit_transform(X_.values)
Y = scaler_y.fit_transform(Y_.values)#.toarray()

In [4]:
X_train = X[:120, :]
X_test = X[121:, :]
y_train = Y[:120, :]
y_test = Y[121:, :]

In [5]:
input_size = X_train.shape[1]

In [6]:
hidden_size = 20 # parameter to tune - no of hidden units

In [7]:
# input_weights = np.random.normal(size=[input_size,hidden_size])
# biases = np.random.normal(size=[hidden_size])
input_weights = np.load(r"C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\elm_input_weights_2_1_20.npy")
biases = np.load(r"C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\elm_biases_2_1_20.npy")

In [8]:
def relu(x):
    return np.maximum(x, 0, x)

In [9]:
def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + biases
    H = relu(G)
    return H

output_weights = np.dot(pinv2(hidden_nodes(X_train)), y_train)

In [10]:
def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

In [11]:
# print(X_test.shape)
# prediction = predict(X_test)
# pred = scaler_y.inverse_transform(prediction)
# actual = scaler_y.inverse_transform(y_test)

print(X.shape)
prediction = predict(X)
pred = scaler_y.inverse_transform(prediction)
actual = scaler_y.inverse_transform(Y)

# result = pd.DataFrame(np.append(actual, pred, scaler_x.inverse_transform(X), axis=1))
result = pd.DataFrame(np.concatenate([actual, pred, scaler_x.inverse_transform(X)], axis=1))

print(result)
print(result.shape)

# test_error = mean_squared_error(actual, pred)
# test_error
print("Mean squared error:", mean_squared_error(actual[:, 0], pred[:, 0]))
print("Mean squared error:", mean_squared_error(actual[:, 1], pred[:, 1]))
print("Mean squared error:", mean_squared_error(actual[:, 2], pred[:, 2]))

(131, 13)
            0          1         2         3          4         5         6  \
0    0.186646  91.275503  7.320161  0.193330  91.299424  7.160471  3.025501   
1    0.158204  91.042365  7.482942  0.173952  91.170807  7.320874  3.025501   
2    0.211328  91.027887  7.586356  0.153414  91.026617  7.502508  3.025501   
3    0.088008  90.827035  7.762458  0.134417  90.882624  7.682705  3.025501   
4    0.103785  90.869638  7.833571  0.123548  90.767262  7.817356  3.025501   
..        ...        ...       ...       ...        ...       ...       ...   
126  0.111707  90.667962  8.103729  0.110160  90.686961  8.060332  3.053684   
127  0.126629  90.389588  8.281747  0.129005  90.610595  8.133283  3.053684   
128  0.143911  90.398711  8.354857  0.147991  90.533541  8.207514  3.053684   
129  0.178287  90.096464  8.515829  0.168314  90.411012  8.347966  3.053684   
130  0.181692  90.204390  8.585448  0.189121  90.272032  8.512373  3.053684   

            7        8         9        1

In [31]:
metrics_df = pd.DataFrame(columns=['metric', 'tripple', 'efficiency', 'tav'])

from sklearn.metrics import r2_score
# R^2
print("R2 tripple", r2_score(actual[:, 0], pred[:, 0]))
print("R2 efficiency", r2_score(actual[:, 1], pred[:, 1]))
print("R2 tav", r2_score(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'R2',
            'tripple': r2_score(actual[:, 0], pred[:, 0]),
            'efficiency': r2_score(actual[:, 1], pred[:, 1]),
            'tav': r2_score(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

def relative_squared_error(y_true, y_pred):
    return np.sum((y_true - y_pred)**2) / np.sum((y_pred - np.mean(y_true))**2)

print("RSE tripple", relative_squared_error(actual[:, 0], pred[:, 0]))
print("RSE efficiency", relative_squared_error(actual[:, 1], pred[:, 1]))
print("RSE tav", relative_squared_error(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'RSE',
            'tripple': relative_squared_error(actual[:, 0], pred[:, 0]),
            'efficiency': relative_squared_error(actual[:, 1], pred[:, 1]),
            'tav': relative_squared_error(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

from sklearn.metrics import mean_absolute_error
print("MAE tripple", mean_absolute_error(actual[:, 0], pred[:, 0]))
print("MAE efficiency", mean_absolute_error(actual[:, 1], pred[:, 1]))
print("MAE tav", mean_absolute_error(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'MAE',
            'tripple': mean_absolute_error(actual[:, 0], pred[:, 0]),
            'efficiency': mean_absolute_error(actual[:, 1], pred[:, 1]),
            'tav': mean_absolute_error(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

def relative_abs_error(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_pred - np.mean(y_true)))

print("RAE tripple", relative_abs_error(actual[:, 0], pred[:, 0]))
print("RAE efficiency", relative_abs_error(actual[:, 1], pred[:, 1]))
print("RAE tav", relative_abs_error(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'RAE',
            'tripple': relative_abs_error(actual[:, 0], pred[:, 0]),
            'efficiency': relative_abs_error(actual[:, 1], pred[:, 1]),
            'tav': relative_abs_error(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

from sklearn.metrics import mean_squared_error
print("MSE tripple", mean_squared_error(actual[:, 0], pred[:, 0]))
print("MSE efficiency", mean_squared_error(actual[:, 1], pred[:, 1]))
print("MSE tav", mean_squared_error(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'MSE',
            'tripple': mean_squared_error(actual[:, 0], pred[:, 0]),
            'efficiency': mean_squared_error(actual[:, 1], pred[:, 1]),
            'tav': mean_squared_error(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

import math
print("RMSE tripple", math.sqrt(mean_squared_error(actual[:, 0], pred[:, 0])))
print("RMSE efficiency", math.sqrt(mean_squared_error(actual[:, 1], pred[:, 1])))
print("RMSE tav", math.sqrt(mean_squared_error(actual[:, 2], pred[:, 2])))
metrics_df = metrics_df.append({'metric': 'RMSE',
            'tripple': math.sqrt(mean_squared_error(actual[:, 0], pred[:, 0])),
            'efficiency': math.sqrt(mean_squared_error(actual[:, 1], pred[:, 1])),
            'tav': math.sqrt(mean_squared_error(actual[:, 2], pred[:, 2]))}, ignore_index=True)
print()

def nrmse_mean(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred)) / np.mean(y_true)

print("NRMSE-mean tripple", nrmse_mean(actual[:, 0], pred[:, 0]))
print("NRMSE-mean efficiency", nrmse_mean(actual[:, 1], pred[:, 1]))
print("NRMSE-mean tav", nrmse_mean(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'NRMSE-mean',
            'tripple': nrmse_mean(actual[:, 0], pred[:, 0]),
            'efficiency': nrmse_mean(actual[:, 1], pred[:, 1]),
            'tav': nrmse_mean(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

def nrmse_min_max(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))

print("NRMSE-min_max tripple", nrmse_min_max(actual[:, 0], pred[:, 0]))
print("NRMSE-min_max efficiency", nrmse_min_max(actual[:, 1], pred[:, 1]))
print("NRMSE-min_max tav", nrmse_min_max(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'NRMSE-min_max',
            'tripple': nrmse_min_max(actual[:, 0], pred[:, 0]),
            'efficiency': nrmse_min_max(actual[:, 1], pred[:, 1]),
            'tav': nrmse_min_max(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

def nrmse_sd(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred)) / np.std(y_true)

print("NRMSE-sd tripple", nrmse_sd(actual[:, 0], pred[:, 0]))
print("NRMSE-sd efficiency", nrmse_sd(actual[:, 1], pred[:, 1]))
print("NRMSE-sd tav", nrmse_sd(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'NRMSE-sd',
            'tripple': nrmse_sd(actual[:, 0], pred[:, 0]),
            'efficiency': nrmse_sd(actual[:, 1], pred[:, 1]),
            'tav': nrmse_sd(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

def nrmse_iq(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred)) / (np.percentile(y_true, 75, interpolation = 'midpoint') - np.percentile(y_true, 25, interpolation = 'midpoint'))

print("NRMSE-iq tripple", nrmse_iq(actual[:, 0], pred[:, 0]))
print("NRMSE-iq efficiency", nrmse_iq(actual[:, 1], pred[:, 1]))
print("NRMSE-iq tav", nrmse_iq(actual[:, 2], pred[:, 2]))
metrics_df = metrics_df.append({'metric': 'NRMSE-iq',
            'tripple': nrmse_iq(actual[:, 0], pred[:, 0]),
            'efficiency': nrmse_iq(actual[:, 1], pred[:, 1]),
            'tav': nrmse_iq(actual[:, 2], pred[:, 2])}, ignore_index=True)
print()

print(metrics_df)
metrics_df.to_csv("elm_metrics.csv")

R2 tripple 0.3067910831184314
R2 efficiency 0.9224246124359717
R2 tav 0.9490572577166386

RSE tripple 2.345956289151328
RSE efficiency 0.08357301382961949
RSE tav 0.05177391155946142

MAE tripple 0.02818962232500978
MAE efficiency 0.08249500448058125
MAE tav 0.07731333191026372

RAE tripple 1.5426124642266204
RAE efficiency 0.2705338327824765
RAE tav 0.21850440559845072

MSE tripple 0.0012181754667778136
MSE efficiency 0.010874319876759738
MSE tav 0.009441001697444726

RMSE tripple 0.03490237050370381
RMSE efficiency 0.10428000708074266
RMSE tav 0.09716481717908353

NRMSE-mean tripple 0.22244163983208823
NRMSE-mean efficiency 0.0011487977077573213
NRMSE-mean tav 0.01192673534915242

NRMSE-min_max tripple 0.17752172989851378
NRMSE-min_max efficiency 0.07280861908817071
NRMSE-min_max tav 0.05515121914301417

NRMSE-sd tripple 0.8325916867718346
NRMSE-sd efficiency 0.2785235852922124
NRMSE-sd tav 0.2257049894959378

NRMSE-iq tripple 0.5547371478469556
NRMSE-iq efficiency 0.1764957878740537

In [ ]:
result.to_csv("elm_result_2_1_20.csv")

In [107]:
# hidden_size = 20 saved as elm_input_weights_2_1_20
#           0          1         2         3          4         5
# 0  0.157900  91.042594  7.484215  0.184395  91.253666  7.398771
# 1  0.211904  91.028190  7.587714  0.165840  91.132588  7.545949
# 2  0.088861  90.827742  7.765061  0.147285  91.011510  7.693127
# 3  0.104405  90.865288  7.836106  0.128730  90.890432  7.840305
# 4  0.149177  90.623488  8.021199  0.110026  90.767623  7.990065
# 5  0.111707  90.667962  8.103729  0.110160  90.686961  8.060332
# 6  0.126629  90.389588  8.281747  0.129005  90.610595  8.133283
# 7  0.143911  90.398711  8.354857  0.147991  90.533541  8.207514
# 8  0.178287  90.096464  8.515829  0.168314  90.411012  8.347966
# 9  0.181692  90.204390  8.585448  0.189121  90.272032  8.512373
# Mean squared error: 0.0008541112384930315
# Mean squared error: 0.028152847780020158
# Mean squared error: 0.009435921771001754

# hidden_size = 15
#           0          1         2         3          4         5
# 0  0.157900  91.042594  7.484215  0.150748  91.142298  7.308853
# 1  0.211904  91.028190  7.587714  0.138120  91.015096  7.461306
# 2  0.088861  90.827742  7.765061  0.126651  90.894056  7.612911
# 3  0.104405  90.865288  7.836106  0.115183  90.773015  7.764517
# 4  0.149177  90.623488  8.021199  0.103714  90.651975  7.916122
# 5  0.111707  90.667962  8.103729  0.093597  90.530909  8.078450
# 6  0.126629  90.389588  8.281747  0.113551  90.430577  8.189894
# 7  0.143911  90.398711  8.354857  0.135675  90.330323  8.293651
# 8  0.178287  90.096464  8.515829  0.157799  90.230068  8.397408
# 9  0.181692  90.204390  8.585448  0.179283  90.132645  8.488816
# Mean squared error: 0.001009891393902576
# Mean squared error: 0.007197333874781289
# Mean squared error: 0.012223004764162191

# hidden_size = 20
#           0          1         2         3          4         5
# 0  0.157900  91.042594  7.484215  0.150277  91.082624  7.466514
# 1  0.211904  91.028190  7.587714  0.149931  90.975553  7.594272
# 2  0.088861  90.827742  7.765061  0.149585  90.868482  7.722030
# 3  0.104405  90.865288  7.836106  0.149239  90.761412  7.849788
# 4  0.149177  90.623488  8.021199  0.149268  90.652518  7.975870
# 5  0.111707  90.667962  8.103729  0.149373  90.543246  8.101604
# 6  0.126629  90.389588  8.281747  0.149479  90.433974  8.227337
# 7  0.143911  90.398711  8.354857  0.149585  90.324703  8.353071
# 8  0.178287  90.096464  8.515829  0.149691  90.215431  8.478805
# 9  0.181692  90.204390  8.585448  0.149797  90.106159  8.604539
# Mean squared error: 0.0013404439585142098
# Mean squared error: 0.006446933441147993
# Mean squared error: 0.0009153300465136139

# hidden_size = 25
#           0          1         2         3          4         5
# 0  0.157900  91.042594  7.484215  0.195983  91.176903  7.389742
# 1  0.211904  91.028190  7.587714  0.180996  91.063698  7.527187
# 2  0.088861  90.827742  7.765061  0.173558  90.951079  7.671584
# 3  0.104405  90.865288  7.836106  0.168162  90.838618  7.817861
# 4  0.149177  90.623488  8.021199  0.170986  90.729177  7.947774
# 5  0.111707  90.667962  8.103729  0.185115  90.624402  8.053058
# 6  0.126629  90.389588  8.281747  0.199244  90.519627  8.158342
# 7  0.143911  90.398711  8.354857  0.213372  90.414852  8.263626
# 8  0.178287  90.096464  8.515829  0.227501  90.310077  8.368910
# 9  0.181692  90.204390  8.585448  0.241630  90.205302  8.474194
# Mean squared error: 0.0035620883273463797
# Mean squared error: 0.011109235874876316
# Mean squared error: 0.008713258738993762

# hidden_size = 50
#           0          1         2         3          4         5
# 0  0.211904  91.028190  7.587714  0.127279  91.066422  7.592496
# 1  0.088861  90.827742  7.765061  0.099261  90.966382  7.732955
# 2  0.104405  90.865288  7.836106  0.095247  90.850357  7.870460
# 3  0.149177  90.623488  8.021199  0.088943  90.735087  8.005373
# 4  0.111707  90.667962  8.103729  0.095238  90.637305  8.099936
# 5  0.126629  90.389588  8.281747  0.112020  90.536524  8.190009
# 6  0.143911  90.398711  8.354857  0.137515  90.396915  8.292283
# 7  0.178287  90.096464  8.515829  0.163337  90.255854  8.395014
# 8  0.181692  90.204390  8.585448  0.189228  90.137184  8.502935
# 0.0013097159674941184
# 0.009534968676056695
# 0.0040260847295648695
input_weights_final = input_weights
biases_final = biases
np.save(r"C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\elm_input_weights_2_1_%d.npy"%hidden_size, input_weights_final)
np.save(r"C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\elm_biases_2_1_%d.npy"%hidden_size, biases_final)

In [22]:
# input_weights_final = input_weights
# biases_final = biases
# np.save(r"C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\elm_input_weights_%d.npy"%hidden_size, input_weights_final)
# np.save(r"C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\elm_biases_%d.npy"%hidden_size, biases_final)

# class Objective(object):
#     def __init__(self):
#         self.grnns = grnns
    
#     def __call__(self, X): # simple addition
#         tav = 0
#         eff = 1
#         tripple = 2
#         value = 0
#         X = np.expand_dims(X, 0)
#         for i, grnn in enumerate(grnns):
#             x_scaled = grnn.scaler_x.transform(X)
#             pred = grnn.nn.predict(x_scaled)
#             pred = np.squeeze(pred, -1)
#             pred = grnn.scaler_y.inverse_transform(pred)
#             if i == tav: # increase
#                 value += (1/pred)
#             elif i == eff: # increase
#                 value += (1/pred)
#             elif i == tripple: # decrease
#                 value += pred
# #         print(value[0][0])
#         return value[0][0]

from scipy.optimize import basinhopping

def objective(X):
    tav = 0
    eff = 1
    tripple = 2
    value = 0
    X = np.expand_dims(X, 0)
    pred = predict(X)
    print("actual before scaling", (pred[0][tav]), (pred[0][eff]), (pred[0][tripple]))
    pred = scaler_y.inverse_transform(pred)
    value = (1/pred[0][tav]) + (1/pred[0][eff]) + (pred[0][tripple])
    value = (10-pred[0][tav]) + (100 - 100*pred[0][eff]) + (pred[0][tripple])
    print("actual after scaling", (pred[0][tav]), (pred[0][eff]), (pred[0][tripple]))
    print(value, (10-pred[0][tav]), (100 - 100*pred[0][eff]), (pred[0][tripple]))
    return value


class MyBounds(object):
    def __init__(self, xmax, xmin):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin
    
def print_fun(x, f, accepted):
    print("at minimum %.4f accepted %d" % (f, int(accepted)))
    
class MyTakeStep(object):
    def __init__(self, stepsize=0.02):
        self.stepsize = stepsize
    def __call__(self, x):
        s = self.stepsize
        x[:-1] += np.random.uniform(s, s)
        x[-1] += np.random.uniform(-0.1*s, 0.1*s, x[-1].shape)
        return x
    
mybounds = MyBounds(xmax=[2.985700615, 4.152054319, 5.20306143, 2.6, 1.458841307, 1.247463957, 10.7643, 24.5952, 36.6826, 0.45], 
         xmin=[3.994173672, 5.229702309, 6.362200008, 6.528583919, 3.133207987, 2.437445577, 13.3, 25.7294, 38.85875, 0.55])

mytakestep = MyTakeStep()

func = objective

x0 = [2.985700615, 4.228527946, 5.20306143, 3.336434202, 1.825015455, 1.247968206, 12.3682, 25.1489, 38.2854 ,0.45]
# x0 = [3.003993443,4.200628853,5.240994991,3.253454486,1.800445199,1.325251093,12.42675,25.3537,38.51655,0.55]
# x0 = np.random.uniform([2.985700615, 4.152054319, 5.20306143, 2.6, 1.458841307, 1.247463957, 10.7643, 24.5952, 36.6826, 0.45],
#                        [3.994173672, 5.229702309, 6.362200008, 6.528583919, 3.133207987, 2.437445577, 13.3, 25.7294, 38.85875, 0.55])
x0 = list(x0)
minimizer_kwargs = {"method": "BFGS"}
# minimizer_kwargs = {"method":"L-BFGS-B"}

ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=1, take_step=mytakestep, accept_test=mybounds, callback=print_fun)

# grnns[0].nn.predict([[ 3.0039935,  4.2006288,  5.240995,   3.2534544,  1.8004452,  1.3252511,
#   12.42675,   25.3537,    38.51655,    0.55     ]])

actual before scaling -12.288751328929992 1.904171398465515 20.733072492053005
actual after scaling -15.33667705222939 0.9216592336196093 415.56734595143314
448.7380996417016 25.33667705222939 7.834076638039065 415.56734595143314
actual before scaling -12.288751328929992 1.904171398465515 20.733072492053005
actual after scaling -15.33667705222939 0.9216592336196093 415.56734595143314
448.7380996417016 25.33667705222939 7.834076638039065 415.56734595143314
actual before scaling -12.28875133395199 1.9041714148078661 20.73307250901543
actual after scaling -15.33667706107709 0.9216592338749509 415.56734628492967
448.73809995851167 25.33667706107709 7.834076612504916 415.56734628492967
actual before scaling -12.288751324109437 1.9041714069357543 20.73307250474016
actual after scaling -15.336677043736591 0.9216592337519528 415.56734620087406
448.73809986941535 25.336677043736593 7.834076624804723 415.56734620087406
actual before scaling -12.288751323468567 1.9041714107962542 20.7330725018285

actual after scaling 17.817574528235383 0.7671293164852484 81.48974625410465
96.95924007734442 -7.817574528235383 23.287068351475156 81.48974625410465
actual before scaling 6.529761867667526 -7.986043955770508 3.7410938555060613
actual after scaling 17.817574526290386 0.7671293165619648 81.48974632837995
96.95924014589309 -7.817574526290386 23.287068343803526 81.48974632837995
actual before scaling 6.529761872352681 -7.986043974654498 3.7410938585833824
actual after scaling 17.817574534544644 0.7671293162669114 81.48974638888284
96.95924022764704 -7.817574534544644 23.287068373308855 81.48974638888284
actual before scaling 6.5297618578328995 -7.986043962754005 3.7410938620556293
actual after scaling 17.81757450896385 0.767129316452851 81.48974645715035
96.9592403029014 -7.817574508963851 23.287068354714904 81.48974645715035
actual before scaling 6.5297618556746055 -7.986043949978219 3.7410938694408062
actual after scaling 17.81757450516139 0.7671293166524665 81.48974660234957
96.959240

actual before scaling 69.07285010405748 -21.131973033433965 -32.06701203085524
actual after scaling 128.00530616184517 0.5617304190397852 -622.5298571234475
-696.7082051892712 -118.00530616184517 43.826958096021485 -622.5298571234475
actual before scaling 69.07285008804324 -21.131973025844143 -32.067012052728096
actual after scaling 128.00530613363145 0.5617304191583725 -622.5298575534877
-696.7082056029564 -118.00530613363145 43.82695808416275 -622.5298575534877
actual before scaling 140.22788393532406 -49.096068402667555 -138.423871276073
actual after scaling 253.36546974514215 0.12480471184081066 -2713.601473919563
-2869.447414848786 -243.36546974514215 87.51952881591893 -2713.601473919563
actual before scaling 140.22788393532406 -49.096068402667555 -138.423871276073
actual after scaling 253.36546974514215 0.12480471184081066 -2713.601473919563
-2869.447414848786 -243.36546974514215 87.51952881591893 -2713.601473919563
actual before scaling 140.22788393666505 -49.09606840442343 -138

actual after scaling 33025.03541630264 -89.50402096381609 -380500.4530444971
-404465.08636441815 -33015.03541630264 9050.40209638161 -380500.4530444971
actual before scaling 18741.585899912734 -5785.5152987893125 -19353.558673186377
actual after scaling 33025.03541630264 -89.50402096381609 -380500.4530444971
-404465.08636441815 -33015.03541630264 9050.40209638161 -380500.4530444971
actual before scaling 18741.585899913378 -5785.515298791908 -19353.558673187104
actual after scaling 33025.03541630378 -89.50402096385665 -380500.45304451144
-404465.08636442956 -33015.03541630378 9050.402096385664 -380500.45304451144
actual before scaling 18741.58589990234 -5785.515298783728 -19353.55867317522
actual after scaling 33025.03541628433 -89.50402096372883 -380500.4530442778
-404465.0863641892 -33015.03541628433 9050.402096372884 -380500.4530442778
actual before scaling 18741.585899912636 -5785.515298778946 -19353.558673173247
actual after scaling 33025.03541630247 -89.50402096365411 -380500.4530

actual after scaling 33559060.83272292 -91940.1516602086 -386537861.4109416
-410902797.07764363 -33559050.83272292 9194115.16602086 -386537861.4109416
actual before scaling 19048281.29929146 -5884405.674263282 -19660232.310033653
actual after scaling 33559060.832722984 -91940.1516602085 -386537861.4109416
-410902797.07764375 -33559050.832722984 9194115.16602085 -386537861.4109416
actual before scaling 19048281.299291432 -5884405.674263285 -19660232.310033645
actual after scaling 33559060.83272293 -91940.15166020855 -386537861.4109415
-410902797.0776436 -33559050.83272293 9194115.166020855 -386537861.4109415
actual before scaling 19048281.299291436 -5884405.674263287 -19660232.31003366
actual after scaling 33559060.83272294 -91940.15166020859 -386537861.4109418
-410902797.0776439 -33559050.83272294 9194115.166020859 -386537861.4109418
actual before scaling 19048281.299291447 -5884405.674263291 -19660232.31003364
actual after scaling 33559060.832722954 -91940.15166020865 -386537861.41094

...

actual after scaling 709075477.2548662 -4497987.632522469 783608236.1440834
524331632.14146405 -709075467.2548662 449798863.2522469 783608236.1440834
actual before scaling 402474659.26628655 -287880017.1160358 39856172.0035997
actual after scaling 709075477.2548662 -4497987.63252247 783608236.1440834
524331632.1414642 -709075467.2548662 449798863.25224704 783608236.1440834
actual before scaling 402474659.2662865 -287880017.11603576 39856172.00359976
actual after scaling 709075477.2548661 -4497987.632522469 783608236.1440846
524331632.14146537 -709075467.2548661 449798863.2522469 783608236.1440846
actual before scaling 402474659.2662865 -287880017.11603576 39856172.00359976
actual after scaling 709075477.2548661 -4497987.632522469 783608236.1440846
524331632.14146537 -709075467.2548661 449798863.2522469 783608236.1440846
actual before scaling 409424712.39251584 -291193213.6286268 -300948732.4465633
actual after scaling 721320005.1526784 -4549754.754263361 -5916923047.104503
-6183267466.

actual before scaling 409900538.25074375 -291442532.7618807 -324247314.76518506
actual after scaling 722158309.9670091 -4553650.247293864 -6374994153.602022
-6641787328.839644 -722158299.9670091 455365124.72938645 -6374994153.602022
actual before scaling 409900606.6135707 -291442568.5820458 -324250662.11784035
actual after scaling 722158430.407893 -4553650.80696693 -6375059965.569191
-6641853205.280391 -722158420.407893 455365180.69669294 -6375059965.569191
actual before scaling 409900606.6135707 -291442568.5820458 -324250662.11784035
actual after scaling 722158430.407893 -4553650.80696693 -6375059965.569191
-6641853205.280391 -722158420.407893 455365180.69669294 -6375059965.569191
actual before scaling 409900606.6135707 -291442568.5820458 -324250662.11784035
actual after scaling 722158430.407893 -4553650.80696693 -6375059965.569191
-6641853205.280391 -722158420.407893 455365180.69669294 -6375059965.569191
actual before scaling 409900606.6135707 -291442568.5820458 -324250662.11784035
a

actual after scaling 722158435.5144906 -4553650.830696621 -6375062755.944102
-6641855998.388931 -722158425.5144906 455365183.0696621 -6375062755.944102
actual before scaling 409900609.52316743 -291442570.1065911 -324250804.5848285
actual after scaling 722158435.5339888 -4553650.830787226 -6375062766.598456
-6641856009.053722 -722158425.5339888 455365183.07872254 -6375062766.598456
actual before scaling 409900609.52316743 -291442570.1065911 -324250804.5848285
actual after scaling 722158435.5339888 -4553650.830787226 -6375062766.598456
-6641856009.053722 -722158425.5339888 455365183.07872254 -6375062766.598456
actual before scaling 409900609.52316743 -291442570.1065911 -324250804.5848284
actual after scaling 722158435.5339888 -4553650.830787226 -6375062766.5984535
-6641856009.0537195 -722158425.5339888 455365183.07872254 -6375062766.5984535
actual before scaling 409900609.52316743 -291442570.1065911 -324250804.5848285
actual after scaling 722158435.5339888 -4553650.830787226 -6375062766.

actual before scaling 409900609.52395827 -291442570.10700554 -324250804.6235601
actual after scaling 722158435.535382 -4553650.830793701 -6375062767.359954
-6641856009.815966 -722158425.535382 455365183.07937014 -6375062767.359954
actual before scaling 409900609.5239582 -291442570.10700554 -324250804.6235601
actual after scaling 722158435.535382 -4553650.830793701 -6375062767.359954
-6641856009.815966 -722158425.535382 455365183.07937014 -6375062767.359954
actual before scaling 409900609.5239583 -291442570.10700554 -324250804.6235601
actual after scaling 722158435.5353822 -4553650.830793701 -6375062767.359954
-6641856009.815966 -722158425.5353822 455365183.07937014 -6375062767.359954
actual before scaling 409900609.52395827 -291442570.10700554 -324250804.6235602
actual after scaling 722158435.535382 -4553650.830793701 -6375062767.359956
-6641856009.815968 -722158425.535382 455365183.07937014 -6375062767.359956
actual before scaling 409900609.52395827 -291442570.10700554 -324250804.6235

actual after scaling 266031841.02111745 -738323.3083941836 -3070594164.1355424
-3262793564.3172417 -266031831.02111745 73832430.83941837 -3070594164.1355424
actual before scaling 151000953.23532107 -47254185.34653972 -156177698.33183646
actual after scaling 266031841.02111727 -738323.3083941835 -3070594164.1355433
-3262793564.317242 -266031831.02111727 73832430.83941835 -3070594164.1355433
actual before scaling 151994372.6411811 -48130167.31282766 -157508435.36240625
actual after scaling 267782036.51003733 -752010.1105259986 -3096757652.3561597
-3289338567.813597 -267782026.51003733 75201111.05259986 -3096757652.3561597
actual before scaling 151994372.6411811 -48130167.31282766 -157508435.36240625
actual after scaling 267782036.51003733 -752010.1105259986 -3096757652.3561597
-3289338567.813597 -267782026.51003733 75201111.05259986 -3096757652.3561597
actual before scaling 151994372.6411811 -48130167.31282766 -157508435.36240625
actual after scaling 267782036.51003733 -752010.1105259986

In [1]:
import pandas as pd
import scipy.io as sio

mat = sio.loadmat(r'12m_P2_Cooling trials_15_03_2020_M0.mat')

In [2]:
mat_1 = {key: mat[key].T.tolist()[0] for key in mat.keys()}

In [23]:
len(mat_1['Time'])

In [4]:
df = pd.DataFrame.from_dict(mat_1)

In [5]:
df

,Time,Ambient_Temperature,Battery_Coolant_Out_Temperature,BMS_Tmin_SubPack_ID,BMS_Tmin_Cell_ID,BMS_Tmax_SubPack_ID,BMS_Tmax_Cell_ID,BMS_HighestOutletTemp,BMS_HighestInletTemp,BMS_MinCellTemp,...,CompressorError,CompressorVoltage,PumpActualSpeed,ActCompressorRPM,BCSDiagnosticsSignal,BMS_SOC,BMS_Total_KwH_In,BMS_Total_KwH_Out,ODO_Meter,EVCU_VehicleSpeed
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67330,67330.0,24.8,30.0,1.0,5.0,5.0,11.0,-30.0,-30.0,18.0,...,0.0,640.0,0.0,0.0,0.0,23.0,2212.2,2140.0,39577.75,0.0
67331,67331.0,24.8,30.0,1.0,5.0,5.0,11.0,-30.0,-30.0,18.0,...,0.0,640.0,0.0,0.0,0.0,23.0,2212.2,2140.0,39577.75,0.0
67332,67332.0,24.8,30.0,1.0,5.0,5.0,11.0,-30.0,-30.0,17.0,...,0.0,640.0,0.0,0.0,0.0,23.0,2212.2,2140.0,39577.75,0.0
67333,67333.0,24.8,30.0,1.0,5.0,5.0,11.0,-30.0,-30.0,17.0,...,0.0,640.0,0.0,0.0,0.0,23.0,2212.2,2140.0,39577.75,0.0


In [7]:
df.to_csv("12m_P2_Cooling trials_15_03_2020_M0.csv")

In [11]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("Heat calculation.xlsx")

In [8]:
df = df.drop([0, 1, 2], axis=0)

In [13]:
df['Unnamed: 0'] = df['Unnamed: 0'].apply(np.ceil)#.round(decimals = 0)

In [16]:
new = df.drop_duplicates("Unnamed: 0")

In [17]:
new.to_csv("Heat calculation 1sec.csv")

In [2]:
ord("'")

39